<a href="https://colab.research.google.com/github/surendrallam/Caffe_Mobilenet_SSD_Custom_Dataset_Training/blob/main/Caffe_Mobilenet_SSD_Custom_Dataset_Training_on_GPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Caffe Installation

In [ ]:
!apt install -y caffe-cuda
!apt build-dep caffe-cuda

Reading package lists... Done
Building dependency tree       
Reading state information... Done
caffe-cuda is already the newest version (1.0.0-6build1).
0 upgraded, 0 newly installed, 0 to remove and 40 not upgraded.
Reading package lists... Done
Picking 'caffe-contrib' as source package instead of 'caffe-cuda'
E: Unable to find a source package for caffe-cuda


In [ ]:
!nvidia-smi
!nvcc --version

Thu Aug  5 13:59:52 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   67C    P8    33W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------


## Installing necessary dependencies.


In [ ]:
!apt-get install -y --no-install-recommends libboost-all-dev
!apt-get install libprotobuf-dev libleveldb-dev libsnappy-dev libopencv-dev libboost-all-dev libhdf5-serial-dev \
libgflags-dev libgoogle-glog-dev liblmdb-dev protobuf-compiler 
!apt-get install libgflags2.2 
!apt-get install libgflags-dev
!apt-get install libgoogle-glog-dev
!apt-get install libhdf5-100
!apt-get install libhdf5-serial-dev
!apt-get install libhdf5-dev
!apt-get install libhdf5-cpp-100
!apt-get install libprotobuf-dev protobuf-compiler

In [ ]:

!apt-get install libgflags2.2 
!apt-get install libgflags-dev
!apt-get install libgoogle-glog-dev
!apt-get install libhdf5-100
!apt-get install libhdf5-serial-dev
!apt-get install libhdf5-dev
!apt-get install libhdf5-cpp-100
!apt-get install libprotobuf-dev protobuf-compiler


##  Cloning Caffe

In [ ]:
!git clone https://github.com/weiliu89/caffe.git
%cd caffe
!git checkout ssd
%cp Makefile.config.example Makefile.config

## Modifying Makefile

In [ ]:
!sed -i 's/-gencode arch=compute_20/#-gencode arch=compute_20/' Makefile.config #old cuda versions won't compile 
!sed -i 's/\/usr\/local\/include/\/usr\/local\/include \/usr\/include\/hdf5\/serial\//'  Makefile.config #one of the 4 things needed to fix hdf5 issues
!sed -i 's/# OPENCV_VERSION := 3/OPENCV_VERSION := 3/' Makefile.config #We actually use opencv 4.1.2, but it's similar enough to opencv 3.
!sed -i 's/code=compute_61/code=compute_61 -gencode=arch=compute_70,code=sm_70 -gencode=arch=compute_75,code=sm_75 -gencode=arch=compute_75,code=compute_75/' Makefile.config #support for new GPUs

## Installing necessary Python packages.

In [ ]:
!apt-get install python3-pip
!pip install scikit-image protobuf
%cd /content/caffe/python
!for req in $(cat requirements.txt); do pip install $req; done

In [ ]:
%cd /usr/lib/x86_64-linux-gnu/
# %ls -al | grep "libhdf5_serial"
!ln -s /usr/lib/x86_64-linux-gnu/libhdf5_serial.so.100.0.1 /usr/lib/x86_64-linux-gnu/libhdf5.so
!ln -s /usr/lib/x86_64-linux-gnu/libhdf5_serial_hl.so.100.0.0 /usr/lib/x86_64-linux-gnu/libhdf5_hl.so
%env CPATH="/usr/include/hdf5/serial/"
!echo $CPATH

/usr/lib/x86_64-linux-gnu
env: CPATH="/usr/include/hdf5/serial/"
"/usr/include/hdf5/serial/"


## Making Caffe

In [ ]:
%cd /content/caffe
# !make clean
!make all
!echo "Done Make all"
!make test
!echo "Done Make test"
!make runtest
!echo "Done Make runtest"

/content/caffe
PROTOC src/caffe/proto/caffe.proto
CXX .build_release/src/caffe/proto/caffe.pb.cc
CXX src/caffe/solvers/sgd_solver.cpp
CXX src/caffe/solvers/rmsprop_solver.cpp
CXX src/caffe/solvers/nesterov_solver.cpp
CXX src/caffe/solvers/adadelta_solver.cpp
CXX src/caffe/solvers/adagrad_solver.cpp
CXX src/caffe/solvers/adam_solver.cpp
CXX src/caffe/net.cpp
CXX src/caffe/syncedmem.cpp
CXX src/caffe/data_transformer.cpp
CXX src/caffe/data_reader.cpp
CXX src/caffe/parallel.cpp
CXX src/caffe/internal_thread.cpp
CXX src/caffe/layer_factory.cpp
CXX src/caffe/layers/annotated_data_layer.cpp
CXX src/caffe/layers/bias_layer.cpp
CXX src/caffe/layers/euclidean_loss_layer.cpp
CXX src/caffe/layers/permute_layer.cpp
CXX src/caffe/layers/prelu_layer.cpp
CXX src/caffe/layers/log_layer.cpp
CXX src/caffe/layers/detection_evaluate_layer.cpp
CXX src/caffe/layers/base_data_layer.cpp
CXX src/caffe/layers/spp_layer.cpp
CXX src/caffe/layers/eltwise_layer.cpp
CXX src/caffe/layers/contrastive_loss_layer.cpp
CX

In [ ]:
!make test
!echo "Done Make test"

make: *** No rule to make target '/usr/local/cuda/include/thrust/detail/reference.inl', needed by '.build_release/cuda/src/caffe/util/bbox_util.o'.  Stop.
Done Make test


In [ ]:
!make runtest
!echo "Done Make runtest"

Streaming output truncated to the last 5000 lines.
debug: (top_id, top_data_id, blob_id, feat_id)=0,0,0,22; feat = 0.17048353748396045; objective+ = 281.98366901977204; objective- = 281.89166902167938
./include/caffe/test/test_gradient_check_util.hpp:175: Failure
The difference between computed_gradient and estimated_gradient is 4.5999999046358653, which exceeds threshold_ * scale, where
computed_gradient evaluates to 0,
estimated_gradient evaluates to 4.5999999046358653, and
threshold_ * scale evaluates to 0.0091999998092717316.
debug: (top_id, top_data_id, blob_id, feat_id)=0,0,0,23; feat = 0.32361167320050305; objective+ = 281.98366901977204; objective- = 281.89166902167932
./include/caffe/test/test_gradient_check_util.hpp:175: Failure
The difference between computed_gradient and estimated_gradient is 4.5999999046330231, which exceeds threshold_ * scale, where
computed_gradient evaluates to 0,
estimated_gradient evaluates to 4.5999999046330231, and
threshold_ * scale evaluates to 0.

In [ ]:
!make pycaffe
!echo "Done Make pycaffe"

make: *** No rule to make target '/usr/local/cuda/include/thrust/detail/reference.inl', needed by '.build_release/cuda/src/caffe/util/bbox_util.o'.  Stop.
Done Make pycaffe


## Verifying Caffe

In [ ]:
import cv2
print(cv2.__version__)
import caffe

4.1.2


ModuleNotFoundError: ignored

# Mobilenet-SSD traning using Custom dataset 

## Cloning Mobilenet-SSD

In [ ]:
# %cd /content/
# !git clone https://github.com/chuanqi305/MobileNet-SSD.git
%cd /content/MobileNet-SSD/
# !ls

In [ ]:
%cd /content/MobileNet-SSD/
# !chmod +x train.sh
!./train.sh

/content/MobileNet-SSD
I0805 12:50:03.469931 27563 caffe.cpp:217] Using GPUs 0
I0805 12:50:03.492552 27563 caffe.cpp:222] GPU 0: `=�~�
F0805 12:50:03.492671 27563 common.cpp:151] Check failed: error == cudaSuccess (100 vs. 0)  no CUDA-capable device is detected
*** Check failure stack trace: ***
    @     0x7fc2e68340cd  google::LogMessage::Fail()
    @     0x7fc2e6835f33  google::LogMessage::SendToLog()
    @     0x7fc2e6833c28  google::LogMessage::Flush()
    @     0x7fc2e6836999  google::LogMessageFatal::~LogMessageFatal()
    @     0x7fc2e710cf82  caffe::Caffe::SetDevice()
    @     0x55b7c7492a43  (unknown)
    @     0x55b7c748e65a  (unknown)
    @     0x7fc2e5634bf7  __libc_start_main
    @     0x55b7c748f05a  (unknown)
Aborted (core dumped)


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!echo "Mounted Google Drive"

Mounted at /content/gdrive
Mounted Google Drive


In [ ]:
# Copying caffe directory from G-Drive
!cp -r /content/gdrive/MyDrive/caffe /content/

In [ ]:
# Copying caffe directory to G-Drive
!cp -r /content/caffe /content/MobileNet-SSD/ /content/gdrive/MyDrive/

In [ ]:
!cp -r caffe/ /content/gdrive/

cp: cannot stat 'caffe/': No such file or directory


# Reference:
* https://askubuntu.com/questions/960238/nvcc-fatal-unsupported-gpu-architecture-compute-20
* https://stackoverflow.com/questions/37007495/caffe-didnt-see-hdf5-h-when-compiling/44912342
* https://github.com/BVLC/caffe/issues/4621#issuecomment-243124779
* https://colab.research.google.com/github/Huxwell/caffe-colab/blob/main/caffe_details.ipynb#scrollTo=TpMhijZ6-9hy
* https://filip-drapejkowski.medium.com/caffe-in-google-colab-2021-2d730989c849